# ZADANIE 1
#### Podział i odtworzenie sekretu - metoda trywialna

In [361]:
from random import seed
from random import randint
def trivial_secret(s,k,n):
    
    tab = []
    print('Initial secret:',s, sep='\n')

    for _ in range(n-1):
        value = randint(0,k)
        s -= value
        tab.append(value)
    tab.append(s % k)
    print('Fragments:',tab, sep='\n')
    sum = 0
    for si in tab:
        sum += si
    recreated_s = sum % k
    print('Recreated secret:',recreated_s, sep='\n')

In [362]:
trivial_secret(500,1000,4)

Initial secret:
500
Fragments:
[472, 490, 263, 275]
Recreated secret:
500


#### Metoda nie jest bezpieczna dla wartości:
* dla n < 3 - mając tylko dwa fragmenty możemy na podstawie jednego wywnioskować drugi i poznać sekret
* dla małych k - przy nie wielkim k można by zwyczajnie "strzelić" sekret

#### Podstawowe wady trywialnego podziału sekretu:
* w momencie, gdy chcemy dla danego zbioru osób wyznaczyć mniejsze podzbiory, które są w stanie wyznaczyć sekret musimy tworzyć dodatkowe podziały - dla większych zbiorów podziałów może być dziesiątki jeśli nie setki tysięcy

# ZADANIE 2
#### Podział i odtworzenie sekretu - metoda zgodna ze schematem Shamira

In [2]:
def find_inverted_modulo(l, m, p):
    i = 1
    while(True):
        if(l * i % p == m):
            return i
        else:
            i += 1

In [3]:
def find_constant(x, y, p, reqShares):
    sum = 0
    for i in range (0, reqShares):
        l = 1
        m = 1
        for j in range(0, reqShares):
            if(i != j):
                l = l * -x[j]
                m = m * (x[i] - x[j])
        if(y[i] * l / m < 0):
            result = -(abs(y[i]) * find_inverted_modulo(abs(m), abs(l), p) % p)
        else:
            result = abs(y[i]) * find_inverted_modulo(abs(m), abs(l), p) % p
        sum = sum + result
    return sum

In [4]:
import numpy as np
from scipy.interpolate import lagrange
from numpy.polynomial.polynomial import Polynomial
from random import randint
from math import ceil
def shamir_secret(maxShares, reqShares, secret, p, maxRange = 1000):
    coef = []
    shares = []
    x_points = []
    
    print('Initial secret:', secret, sep='\n')

    for _ in range(reqShares-1):
        value = randint(0,maxRange)
        coef.append(value)
        
    print('Coefficients:', coef, sep='\n')
    
    for x in range(1,maxShares+1):
        sum = secret
        power = 1
        for c in coef:
            sum += c*x**power
            power+=1
        x_points.append(x)
        shares.append(sum%p)
        
    print('Shares', shares, sep='\n')
    while (len(shares) != reqShares):
        i = randint(0,len(shares)-1)
        x_points.pop(i)
        shares.pop(i)
        
    print('Coordinates:')
    for i in range(0,reqShares):
        print('(',x_points[i],',',shares[i],')')
        
    constant = find_constant(x_points, shares, p, reqShares)
    
    print('Constant term:',constant,sep='\n')
    a = constant
    
    print('Calculating secret:')
    print(a,'%',p,'=',a%p)
    secret = a % p
    
    print('Recreated secret:', secret, sep='\n')
    

In [15]:
shamir_secret(4,3,954,1523)

Initial secret:
954
Coefficients:
[975, 356]
Shares
[762, 1282, 991, 1412]
Coordinates:
( 2 , 1282 )
( 3 , 991 )
( 4 , 1412 )
Constant term:
954
Calculating secret:
954 % 1523 = 954
Recreated secret:
954


Minimalna liczba udziałów potrzebna do działania algorytmu to 2.